# 5: MDN data generation component

This handles generating new data using the trained MDN deep abstract models produced in 3.

## Step 0: Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from simulation_manager import SimulationManager
from mdn_manager import MdnManager

import pickle
import os
from time import time

In [ ]:
current_dir = os.getcwd()

In [ ]:
def gen_per_config(name, path, data_configs, plot=False):
    simulation_times = dict()
    if not os.path.exists("mdn_datasets"):
        os.makedirs("mdn_datasets")
        
    for case in data_configs:
        config = data_configs[case]
        end_time = config["end_time"]
        n_steps = config["n_steps"]
        n_init_conditions = config["n_init_conditions"]
        n_sims_per_init_condition = config["n_sims_per_init_condition"]
        time_step = end_time / n_steps

        config_name = f"{name}_{n_steps}_{end_time}_{n_init_conditions}_{n_sims_per_init_condition}"

        sm = SimulationManager(
            path_to_sbml=path,
            model_name=name,
            n_init_conditions=n_init_conditions,
            n_sims_per_init_condition=n_sims_per_init_condition,
            end_time=end_time,
            n_steps=n_steps
        )

        mm = MdnManager(
            sm.model_name,
            sm.get_num_species()
        )

        with open(f"ssa_datasets/{config_name}_test.pickle", "rb") as f:
            testing_data = pickle.load(f)

        ic = sm.extract_initial_conditions_from_dataset(testing_data, n_init_conditions)
        ic = sm.add_time_column(ic)

        start_time = time()
        mdn_data = mm.simulate(ic, time_step, n_steps, n_sims_per_init_condition)
        time_taken = time() - start_time

        simulation_times[case] = time_taken

        if plot:
            print("Plotting...")
            sm.plot_simulations(
                f"plots/{config_name}__mdn",
                mdn_data,
                n_init_conditions,
                n_sims_per_init_condition,
                sm.get_column_names()
            )

        with open(f"mdn_datasets/{config_name}.pickle", "wb") as f:
            pickle.dump(mdn_data, f)
        print(f"Generated MDN data of shape {mdn_data.shape}.")

## Step 1: Multifeedback model

In [ ]:
relative_path = "crn_models/1_multifeedback.txt"
path = os.path.join(current_dir, relative_path)
name = "multifeedback"

In [ ]:
gen_config = {
    # depth
    "case_1": {
        "end_time": 32,
        "n_steps": 16,
        "n_init_conditions": 100,
        "n_sims_per_init_condition": 200
    },
    # breadth
    "case_2": {
        "end_time": 32,
        "n_steps": 16,
        "n_init_conditions": 200,
        "n_sims_per_init_condition": 100
    }
}

In [ ]:
gen_per_config(name, path, gen_config, plot=True)

## Step 2: Repressilator model

In [ ]:
relative_path = "crn_models/2_repressilator.txt"
path = os.path.join(current_dir, relative_path)
name = "repressilator"

In [ ]:
gen_config = {
    # breadth
    "case_1": {
        "end_time": 128,
        "n_steps": 32,
        "n_init_conditions": 200,
        "n_sims_per_init_condition": 100
    }
}

In [ ]:
gen_per_config(name, path, gen_config, plot=True)